In [6]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-132307")
exp = Experiment(workspace=ws, name="quick-starts-experiment")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-132307
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-132307


In [10]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# the code sninppet is based on official Microsoft documentation, which can be found
# here -> https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python
from azureml.core.compute_target import ComputeTargetException

# A name for the cluster
cpu_cluster_name = "Udacity-Lab1"

# Verification that cluster does not exist yet
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('The cluster already exists')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

The cluster already exists

Running


In [35]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--C": choice(0.001, 0.01, 0.1, 1.0),
        "--max_iter": choice(30, 50, 100, 250)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=50, slack_factor=None, slack_amount=0.2, delay_evaluation=0)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./", compute_target=cpu_cluster, entry_script="train.py", script_params=None)
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, primary_metric_name="Accuracy", 
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=20, 
                                    max_concurrent_runs=4, policy=policy, estimator=est)

In [36]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_submit = exp.submit(hyperdrive_config)

In [37]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_submit).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [52]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
# the code is based on the official Microsoft documentation, which can be found
# here -> https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters#find-the-best-model 
best_run = hyperdrive_submit.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

Best Run Id:  HD_cedfa4c0-aa60-4094-a198-eddd0392cac3_4

 Accuracy: 0.9118079823432039


In [64]:
best_model = best_run.register_model(model_name="best_model", model_path="./")

In [92]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
datastore_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=datastore_path)
ds.to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,housemaid,married,basic.4y,no,no,yes,cellular,jul,mon,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
32946,37,management,married,university.degree,no,no,yes,cellular,jul,fri,...,7,999,0,nonexistent,1.4,93.918,-42.7,4.957,5228.1,no
32947,26,admin.,single,university.degree,no,no,no,cellular,may,tue,...,4,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
32948,31,blue-collar,single,basic.9y,no,no,no,cellular,apr,mon,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,no


In [95]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
train_dataset = pd.concat([x_train, y_train], axis=1)

In [96]:
train_dataset

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
20260,31,1,0,0,0,8,1,70,1,999,...,0,0,0,0,0,0,0,1,0,0
4030,30,0,0,0,0,8,1,135,1,999,...,0,0,0,0,0,0,0,1,0,0
26303,33,1,0,0,0,6,5,432,1,999,...,1,0,0,1,0,0,0,0,0,0
13348,36,0,0,1,1,5,2,193,5,999,...,1,0,0,0,1,0,0,0,0,0
6984,43,1,0,0,0,5,2,535,1,999,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2226,26,0,0,0,1,11,3,358,1,9,...,0,0,0,0,0,0,1,0,0,1
16942,73,1,0,1,1,8,5,135,1,999,...,0,1,0,0,0,0,0,0,0,0
6462,32,0,0,1,0,8,5,118,5,999,...,0,0,0,0,0,0,0,1,0,0
17573,25,0,0,0,0,8,4,215,1,999,...,0,0,0,0,0,0,0,1,0,1


In [109]:
from azureml.core import Dataset, Datastore

# the code is based on the official Microsoft documentation, which can be found 
# here -> https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-register-datasets#create-a-dataset-from-pandas-dataframe 

local_path = './prepared.csv'
train_dataset.to_csv(local_path)

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='./', target_path='./')

# create a dataset referencing the cloud location
dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('./prepared.csv'))])

Uploading an estimated of 16 files
Uploading ./automl.log
Uploaded ./automl.log, 1 files out of an estimated total of 16
Uploading ./azureml_automl.log
Uploaded ./azureml_automl.log, 2 files out of an estimated total of 16
Uploading ./model.joblib
Uploaded ./model.joblib, 3 files out of an estimated total of 16
Uploading ./train.py
Uploaded ./train.py, 4 files out of an estimated total of 16
Uploading ./udacity-project.ipynb
Uploaded ./udacity-project.ipynb, 5 files out of an estimated total of 16
Uploading ./.ipynb_aml_checkpoints/udacity-project-checkpoint2020-11-29-12-33-13.ipynb
Uploaded ./.ipynb_aml_checkpoints/udacity-project-checkpoint2020-11-29-12-33-13.ipynb, 6 files out of an estimated total of 16
Uploading ./.ipynb_aml_checkpoints/udacity-project-checkpoint2020-11-29-12-57-13.ipynb
Uploaded ./.ipynb_aml_checkpoints/udacity-project-checkpoint2020-11-29-12-57-13.ipynb, 7 files out of an estimated total of 16
Uploading ./.ipynb_aml_checkpoints/udacity-project-checkpoint2020-11-

In [110]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=dataset,
    label_column_name="y",
    n_cross_validations=5,
    compute_target=cpu_cluster)

In [111]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.widgets import RunDetails
automl_submit = exp.submit(automl_config)
RunDetails(automl_submit).show()

Running on remote.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
import joblib

# the code is based on the official Microsoft documentation, which can be found
# here -> https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters#find-the-best-model 
best_automl_run = automl_submit.get_best_run_by_primary_metric()
best_automl_run_metrics = best_automl_run.get_metrics()

print('Best Run Id: ', best_automl_run.id)
print('\n Accuracy:', best_automl_run_metrics['Accuracy'])

best_model = best_run.register_model(model_name="best_automl_model", model_path="./")

In [ ]:
cpu_cluster.delete()